# Settings

In [190]:
IN_FILENAME = "../data/processed/filtered.xlsx"
OUT_FILENAME = "../data/product/product.xlsx"

# adjustments

In [191]:
import re

fmt = re.search("[.][a-z]+$", OUT_FILENAME)
if fmt == None: raise Exception("have to choose a format (Xlsx | csv)")
fmt = fmt.group()[1:]

# Prevs

### Imports

In [192]:
import pandas as pd
import numpy as np
from collections import namedtuple
import json

In [193]:
with open("../constants/columns.json", "r") as file: columns, years = json.load(file).values()
columns = namedtuple("Columns", " ".join(columns.keys()))(**columns)
years = namedtuple("Years", re.sub(r'[\[\],\']', "", str([*map(lambda y: f'y{y}', years)])))(*years)

In [194]:
with open("../constants/filtered_columns.json", "r") as file:
    fcolumns = json.load(file)
    
fcolumns = namedtuple("FColumns", " ".join(fcolumns))(**fcolumns)

### tools

In [195]:
YYYY = lambda txt, year : txt.replace('[YYYY]', str(year))

# Main

In [196]:
data = pd.read_excel(IN_FILENAME)

In [197]:
df = pd.DataFrame(columns=[*fcolumns._asdict().keys()])

for row in data.iloc:
	
	df = pd.concat([
		df, pd.DataFrame([[row[YYYY(col, year)]  for col in fcolumns] for year in years], columns=[*fcolumns._asdict().keys()])
		], ignore_index=True)


C:\Users\nelso\AppData\Local\Temp\ipykernel_20136\4229065489.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([


In [198]:
df.head()

,TD_div_TA,WC_div_TA,EBITDA_div_TA,lnTA,isFamiliar,z_cat
0,0.563004,0.039019,0.050417,19.051693,-1.0,1.0
1,0.565019,0.031774,0.019354,19.030923,-1.0,1.0
2,0.574824,0.041247,0.059132,19.070886,-1.0,1.0
3,0.619957,0.231849,0.043954,19.686780,0.0,2.0
4,0.625639,0.191468,0.030919,19.710178,0.0,2.0


# Save dataframe

In [199]:
match fmt:
    case "xlsx": df.to_excel(OUT_FILENAME, index=False)
    case "csv": df.to_csv(OUT_FILENAME, index=False)
    case _: raise Exception("Stop touching things!!")